In [ ]:
#semantic Segmentation for video 1
import cv2
import numpy as np
import json
from shapely.geometry import Polygon

def json_to_mask(json_file, image_shape):
    # Load JSON file
    with open(json_file) as f:
        data = json.load(f)

    # Create empty mask
    mask = np.zeros(image_shape[:2], dtype=np.uint8)

    # Iterate over labeled objects
    for obj in data['shapes']:
        label = obj['label']
        polygon = Polygon(obj['points'])

        # Create binary mask for each polygon
        cv2.fillPoly(mask, [np.array(polygon.exterior.coords, dtype=np.int32)], 255)

    return mask

# Example usage:
json_file = '/content/drive/MyDrive/Heart/2nd_trimester_images/Fetal Echocardiography Second Trimester/Fetal Echocardiography Second Trimester/Video1 new/3VT(3).json'
image_shape = (512, 512)  # Shape of the image corresponding to the JSON file
mask = json_to_mask(json_file, image_shape)

# Save mask as an image
cv2.imwrite('segmentation_mask1.png', mask)
print("Segmentation mask saved as segmentation_mask.png")

Segmentation mask saved as segmentation_mask.png


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import json
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose

# Load JSON file containing labeled data
def load_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data

# Convert labeled data into image
def json_to_image(data):
    height = data['imageHeight']
    width = data['imageWidth']
    image = np.zeros((height, width), dtype=np.uint8)

    for region in data['shapes']:
        label = region['label']
        contour = np.array(region['points'], dtype=np.int32)
        cv2.fillPoly(image, [contour], (255,))

    return image

# FCN Model
def fcn_model(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Decoder
    x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

# Perform semantic segmentation (FCN)
def semantic_segmentation(image):
    image = np.expand_dims(image, axis=-1)
    model = fcn_model(image.shape)

    # Assuming you have a proper dataset for training the model
    # Here you would load the weights or train the model before making predictions
    # For illustration purposes, we are using a simple thresholding as before.
    _, segmented_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    return segmented_image

# Save segmented images
def save_segmented_images(segmented_images, output_dir, filename):
    for i, segmented_image in enumerate(segmented_images):
        cv2.imwrite(f"{output_dir}/{filename}_segmented.png", segmented_image)

# Main function
def main(input_folder, output_folder):
    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            json_file = os.path.join(input_folder, filename)
            output_dir = os.path.join(output_folder, filename[:-5])  # Remove ".json" extension
            os.makedirs(output_dir, exist_ok=True)

            data = load_json(json_file)
            image = json_to_image(data)
            segmented_image = semantic_segmentation(image)
            save_segmented_images([segmented_image], output_dir, filename[:-5])

if __name__ == "__main__":
    input_folder = "/content/drive/MyDrive/Heart/Raw dataset Fetal Echocardiography Second Trimester/Video7(new)"
    output_folder = "/content/drive/MyDrive/Segmented data/Vid7"
    main(input_folder, output_folder)
